In [1]:
from pyspark import SparkContext

In [2]:
sc = SparkContext()

In [14]:
MEDIA_ID = 3
TS_LISTEN = 2
USER_ID = 12

In [15]:
def sort_and_add_count(lists):
    if len(lists) == 1:
        lists[0].extend(['0', ''])
        return lists
    lists = sorted(lists, key=lambda x: x[TS_LISTEN])
    lists = list(map(lambda x: x[0]+[str(x[1])], zip(lists, range(len(lists)))))
    lists[0].append('')
    lists = [lists[0]] + list(map(lambda x: x[1]+[str(int(x[1][TS_LISTEN])-int(x[0][TS_LISTEN]))], zip(lists[:-1], lists[1:]))) 
    return lists

In [16]:
data = sc.textFile('../data/test_without_header.csv')
out = data.map(lambda x: x.split(',')) \
    .groupBy(lambda x: (x[MEDIA_ID], x[USER_ID])) \
    .flatMap(lambda x: sort_and_add_count(list(x[1]))) \
    .map(lambda x: ','.join(x)) \
    .saveAsTextFile('file:///Users/khan/datasciencegame/data/spark.csv')